# DIffusjon og Random Walk

### TFY4165 Termisk fysikk - Numerisk Øving 3

**Forkunnskaper**: Programmeringskunnskaper tilsvarende det opparbeidet i ITGK, samt noen av ferdighetene opparbeidet i emnene FY1001, FY1003 og TFY4163. I denne øvingen er plotting i både 2 og 3 dimensjoner antatt som kjent.

**Læringsmål**: Undersøke hvordan en mikroskopisk tilnærming til diffusjonsfenomenet skiller seg fra en makroskopisk tilnærming. Den mikroskopiske tilnærmingen vi skal se på omhandler *random walk*, eller virrevandring, ifølge [Norsk matematikkråd sin ordliste](https://matematikkradet.no/ordliste/). Vi holder oss derimot til begrepet "random walk" her. Den makroskopiske tilnærmingen omhandler løsning av diffusjonsligningen. 

Diffusjon er en fysisk prosess der en substans sprer seg fra områder med høy konsentrasjon til områder med lavere konsentrasjon.

Denne øvingen skal brukes til å vise konsekvensene av hvilken tilnærming man velger for beskrivelse av diffusjonsfenomenet. Først skal diffusjonsligningen presenteres, sammen med en løsning av den for et gitt tilfelle. Deretter skal random walk diskuteres. Vi skal se på et 1-dimensjonalt system, men det vi gjør kan utvides til flere dimensjoner.

Et mål er at koden skal skrives såpass generell at denne øvingen kan brukes til å studere andre systemer ved en senere anledning. Dette kan forhåpentligvis komme til nytte i resten av kurset.

In [ ]:
# Import all modules that may be useful in the assignment. 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl # Usecase: rcParams. 
from mpl_toolkits.mplot3d import Axes3D # Usecase: 3D-plotting. 

from scipy.signal import unit_impulse # Usecase: Make a Dirac delta function. 

### Oppgave 1 - Diffusjonsligningen

Den generelle diffusjonsligningen tar formen

$$
    \frac{\partial n}{\partial t} = \nabla^2(Dn), 
$$

der $n$ er konsentrasjonen av den gitte partikkeltypen, og $D$ kalles for diffusjonskonstanten. Denne kan være posisjonsavhengig.

Dersom vi antar posisjonsuavhengig diffusjonskonstant $D$ tar ligningen formen

$$
    \frac{\partial n}{\partial t} = D\nabla^2(n).
$$

Vi skal se på diffusjon på systemet

$$
\begin{align*}
\frac{\partial n}{\partial t} & = \frac{\partial^2 n}{\partial x^2}, && \text{$D=1$, 1D}.\\ \\
n(x,0) & = \delta (x), && \text{Initialverdi}. \\ 
\end{align*}
$$

Det er ikke satt noen begrensninger på hvor langt partiklene kan diffundere (ingen grensebetingelser). 


Startverdien for et 1-dimensjonalt system er plottet nedenfor.

In [ ]:
x = np.linspace(-2, 2, 101) # E.g. 
delta = unit_impulse(101, idx = "mid") # Places 1.0 at the centre of a list with 101 zeros. 

plt.plot(x, delta)
plt.title("Initialverdi (" + r"$\delta (x)$" +r" at $t=0)$")
plt.show()

Den analytiske løsningen til dette systemet er gitt som 

$$
    n(r, t) = \left(\frac{1}{4\pi Dt}\right)^{1/2}\exp{\left(-\frac{r^2}{4Dt}\right)}. 
$$

Du skal jobbe med denne løsningen i hele oppgave 1. 

**a)** Lag et sett med subplots av den analytiske løsningen for ulike tider. Det skal være en rad med 3 ulike $t$-verdier
* t = 0.0001 (Den divergerer i 0, men vi kan gå vilkårlig nærme)
* t = 0.1
* t = 1.0

Dette skal være 2-dimensjonale plott med utstrekning på x-aksen og intensitet på y-aksen. Det skal være navn på aksene, undertitler med hvilken verdi av $t$ som er plottet og en felles super-tittel over hele plottet. Utover dette står du fritt til å selv endre på hvordan du ønsker at den grafiske fremstillingen skal se ut.
Du kan endre på matplotlib.rcParams dersom du ønsker å endre den grafiske framstillingen.

In [ ]:
#mpl.rcParams # Dictionary used to change default attributes of matplotlib.pyplot. 

def analytic_solution(r, t, dim = 1):
    """One possible analytic solution to the diffusion equation.
    
    The initial condition is a Dirac-Delta function at t=0.
    You have to specify the dimension you are working with.
    """
    D = 1 
    return (1/(4*np.pi*D*t))**(dim/2)*np.exp(-r**2/(4*D*t))

#Skriv koden din her.

**b)** Lag et sett med subplots av den analytiske løsningen med 3-dimensjonale overflate-plot, der du har utstrekning ($x$), tid ($t$) og intensitet ($I$) som verdier på aksene. Det skal være 3 suplots fra 3 ulike vinkler, slik at man kan se overflaten fra tre ulike kanter. 

NB: Tenk nøye over hvilke tider du vil ha med i overflateplottet ditt, med tanke på at initialverdien er en delta-funksjon. Hvis man går fra veldig små tider til større tider kan plottet bli litt intetsigende. Eventuelt kan du bruke `numpy.clip` dersom du vil fjerne verdier utenfor et visst intervall. Du kan bruke `help`-funksjonen, eller søke opp i dokumentasjonen til numpy. dersom du ønsker mer informasjon, 

Du trenger altså ikke å plotte for de samme t-verdiene som tidligere, dersom du ikke ønsker.

In [ ]:
# Skriv koden din her.

### Oppgave 2 - Random Walk

#### Simulering ved hjelp av Python

Fram til nå har vi visualisert den analytiske løsningen på flere måter, og er godt kjent med formen på den. Fra nå av skal vi simulere en 1-dimensjonal random walk ved hjelp av pseudo-tilfeldig genererte tall i Python. Til slutt skal de sammenlignes. 

Først en liten kommentar til `random`-modulen og pseudo-tilfeldige tall. Tallene genereres av noe som kalles en *pseudorandom number generator* (PRNG), som er en algoritme som genererer deterministiske sekvenser av tall, som har egenskaper som ligner sterkt på egenskapene til helt tilfeldige tall. PRNG er ofte også kjent som *deterministic random bit generator* (DRBG), som kanskje er et litt mer oppklarende navn. Det som gjør disse sekvensene deterministiske er at de er fullstendig gitt av initialverdien som algoritmen bruker. Denne kalles ofte for *seed*. Det betyr at dersom du bruker den samme PRNG-en med samme seed på forskjellige steder vil de gi den samme sekvensen. Python bruker en PRNG som heter Mersenne Twister, som er en videns kjent algoritme som brukes i veldig mange programvare-systemer. Det finnes derimot metoder som brukes til å lage faktisk tilfeldige tall. Disse kalles ofte *true random number generator* (TRNG) eller *hardware random number generator* (HRNG), siden de bruker fysiske egenskaper ved maskinvare (hardware) for å lage tallene. Men hvorfor bruker man PRNG, dersom man kan lage tilfeldige tall? Årsakene er hastighet og kompleksitet. Det går mye raskere å generere pseudo-tilfeldige tall med en PRNG enn å lage faktisk tilfeldige tall ved hjelp av maskinvare. Det er også mye enklere å implementere en algoritme til å generere tall, enn å bruke maskinvaren. Derimot brukes HRNG på flere områder der en vanlig PRNG gir for deterministiske utfall, som for eksempel i kryptografi. Til de fleste formål i programmering er derimot fordelene ved bruk av PRNG i overtall, noe som er grunnen til at de er såpass utbredte. 

Situasjonen vi skal se på er som følger:

Vi har et gitt antall partikler som alle begynner i $x = 0$. Systemet vi ser på er isolert, uten en ytre påvirkning, som for eksempel et potensial. Da er det rimelig å anta at sannsynligheten for å bevege seg til venstre ($p^-$) og til høyre ($p^+$) er like stor, dvs $p^- \approx p^+ \approx \frac{1}{2}$. Vi trekker et (pseudo-)tilfeldig tall mellom 0 og 1. Dersom tallet er mindre enn 0.5 går partikkelen til venstre. Dersom tallet er større enn, eller lik, 0.5 går partikkelen til høyre. `numpy` har pseudo-random-generatorer som benytter seg av `random`-modulen. 

**a)** Du har 10000 partikler, der alle begynner i $x = 0$. Partiklene skal foreta 25 steg (random walk), med steglengde 0.1. Skriv en kodesnutt som simulerer prosessen, og plott et histogram som viser hvordan partiklene har spredd seg utover etter at simuleringen er over. Se cellen nedenfor for en start på koden. For å redusere kjøretiden skal vi også bruke en modul som heter `numba` i resten av øvinga. Du kan lese mer om hvordan den fungerer i [dokumentasjonen](https://numba.pydata.org/numba-doc/latest/index.html), der deres '[~5 minute guide to Numba](https://numba.pydata.org/numba-doc/latest/user/5minguide.html)\' kan være et greit sted å starte. Du trenger ikke å vite hvordan den fungerer her derimot, siden alt du trenger allerede er lagt til i koden nedenfor. Denne kan gi litt obskure feilmeldinger, slik at det kan være du må lete litt i dokumentasjonen. Men vi satser på å slippe det her. Du **må** ikke bruke den, men den er lagt til for at du forhåpentligvis skal slippe å vente lenge på at koden din skal kjøre ferdig. 

Modulen installerer du med conda. Åpne et terminal-vindu (eller Anaconda-prompt), på samme måte som instruert i en tidligere øving, og skriv `conda install numba`, 

*Cell magic*-kommandoen `%%time`, som du ble kjent med i forrige øving, skal også brukes for å finne kjøretiden til koden din. Vær oppmerksom på at du også kan få litt obskure feilmeldinger når `%%time` brukes.

In [ ]:
%%time

from numba import jit # Use to significantly reduce runtime. 

particles = 10000
time_steps = 25 # # of steps everty particle takes.
step_length = 0.1 # Step length. 
initial = np.zeros(particles) # All particles initiated at position 0. 

p_minus = 1/2
p_plus = 1 - p_minus
bins = 100 # Binning in histogram. 

plt.hist(initial, bins = bins)
plt.title("{} particles, initial condition".format(particles))
plt.ylabel("Particle Quantity")
plt.xlabel("Position")
plt.show()



# Python-decorator: Instructs Numba to compile the code without using Python-interpreter. 
# This is very efficient with loops, but does not work with matplotlib. 
@jit(nopython=True) 
def random_walk_1d(initial = initial): 
    '''Returns a copy of initial array after the simulation.'''
    pos = initial.copy() # Python uses pass by reference. 
    # Without the copy the list 'initial' will be modified.
    
    # Skriv koden din her.
    
    return pos


**b)** Hva skjer dersom man endrer på antall steg som tas? Lag et sett med subplots som inneholder histogrammer for ulikt antall steg

* 10
* 30
* 100
* 2000

In [ ]:
%%time

time_list = [10, 30, 100, 2000]

# Skriv koden din her. 

#### Nå er det på tide å relatere dette tilbake til situasjonen i oppgave 1, slik at vi kan se at de to fremgangsmåtene gir omtrent samme resultat. Men hvordan kan vi sammenligne stegene i random walk-simuleringen med makroskopisk tid?

Vi vet at $D = \frac{h^2}{2t_c} \Leftrightarrow t_c = \frac{h^2}{2D}$, der $h$ er steglengden, $D$ er diffusjonskonstanten og $t_c$ er karakteristisk tid mellom hvert steg. Videre kan vi definere makroskopisk tid som $t = Nt_c$, der $N$ er antall steg (random walks). Alt i alt betyr det dermed at 

$$
    N = \frac{2D t}{h^2}.
$$

Dette er relasjonen mellom antall steg i en random walk og makroskopisk tid. 

**c)** Plott subplott med random walk-histogrammene sammen med de analytiske løsningene fra oppgave 1. Husk å bruke relasjonen ovenfor. 

In [ ]:
%%time

timestamps = [0.1, 0.5, 2.0] # E.g: No reason to plot the exact same timesteps as in task 1. 

# Skriv koden din her. 

Merk: Random walk-tilnærmingen og tilnærmingen via diffusjonsligningen vil være mer og mer like jo flere partikler man bruker og jo kortere steglengde man bruker. Til tross for relativt kort kjøretid, mye takket være `numba`-modulen, kommer likevel poenget fram.

#### Hva sitter vi igjen med fra denne øvingen? 

Vi har sett at diffusjon er et makroskopisk fenomen, som kan behandles som nettopp dette, via diffusjonsligningen. Derimot kan diffusjon også beskrives ved hjelp av mikroskopisk random walk. De to fremgangsmåtene gir en tilnærmet lik beskrivelse av fenomenet, og kan dermed fungere som approksimasjoner til hverandre i en del tilfeller. 

## Ekstra: Kuriositet - lage GIF fra matplotlib-figurer

For de som er interesserte følger det litt informasjon om hvordan du kan lage en GIF via matplotlib og kommandolinje.

Du har nok gjort deg opp noen tanker om begrensningene knyttet til alle subplottene du har laget i løpet av denne øvingen. Det finnes vel bedre, mer givende måter å visualisere slike tidsutviklinger på? Her skal du få en introduksjon til programvare for å lage animasjoner fra stillbilder. 

Vi skal bruke programvare som kalles for `ImageMagick`, som først og fremst brukes via kommandolinja. Dette må installeres. 

### Installasjonsguide for Mac / Linux:
For å unngå konflikter mellom ulik programvare lager vi her en *virtual environment* i conda, som inneholder kun `ImageMagick`-programvaren. Dermed kan vi aktivere denne når vi ønsker å bruke programvaren, men vi har ikke tilgang til programvaren utenfor. Grunnen til at dette er lurt er at vi slipper å forkludre 'resten' av programvaren vår, med programvare som vi sjeldent skal bruke uansett. I tillegg kan vi enkelt slette en slik virtual environment når vi er ferdig, og da forsvinner også programvaren installert der. Dette ble gjennomgått raskt i forrige øving, men det repeteres her. 

1. Åpne et terminalvindu (søk etter "terminal" i søkefeltet)
2. Lag et virtual environment i conda som heter 'ImageMagick_env': 
    > conda create --name ImageMagick_env
3. Aktiver 'miljøet': 
    > conda activate ImageMagick_env
4. Nå som vi er inne i 'miljøet' (du vil se at det står navnet på 'miljøet' foran markøren i terminalen) kan vi installere ImageMagick ved å skrive inn følgende i terminalvinduet: 
    > conda install -c conda-forge imagemagick 


### Installasjonsguide for Windows
Hvis du tok en rask titt i installasjonsguiden for Mac / Linux oppdaget du kanskje at vi der opprettet en virtual environment i conda, for å skille `ImageMagick`-programvaren fra alt annet. Dessverre finnes ikke `ImageMagick` for Windows i conda. Dermed må nedlastningen gjøres direkte fra [ImageMagick sin nettside](https://www.imagemagick.org/script/download.php) i stedet. Velg gjerne den installasjonsfila de anbefaler for ditt Windows-system. For undertegnede fungerte det fint å la alle innstillingene i installasjonsguiden være som de er. Når du er ferdig installert kan du sjekke at alt er som det skal ved å skrive `magick --version` i en Command Prompt. Alt er som det skal dersom du får opp et versjonsnummer. 

Tanken bak denne måten å lage animasjoner på er ganske enkel. Vi lager en løkke som plotter en ny 3D-figur for økende vinkler og lagrer hver figur som en PNG. Deretter skal vi bruke `ImageMagick` til å gjøre om disse bildene til en GIF. Koden jeg har brukt er nedenfor. Store deler av denne koden er tatt fra [dokumentasjonen til matplotlib](https://matplotlib.org/)\.

In [ ]:
%%time
import matplotlib.tri as mtri


# Make a mesh in the space of parameterisation variables u and v
u = np.linspace(0, 2.0 * np.pi, endpoint=True, num=50)
v = np.linspace(-0.5, 0.5, endpoint=True, num=10)
u, v = np.meshgrid(u, v)
u, v = u.flatten(), v.flatten()

# This is the Mobius mapping, taking a u, v pair and returning an x, y, z
# triple
x = (1 + 0.5 * v * np.cos(u / 2.0)) * np.cos(u)
y = (1 + 0.5 * v * np.cos(u / 2.0)) * np.sin(u)
z = 0.5 * v * np.sin(u / 2.0)

# Triangulate parameter space to determine the triangles
tri = mtri.Triangulation(u, v)

# Code taken from Matplotlib-docs as an example. You can plot whatever you want. 
for angle in range(70,210,2): #Løkke fra 70 grader til 210 grader

    # Lager et plott, som vanlig, også tatt fra dokumentasjonen.

    # Plot the surface.  The triangles in parameter space determine which x, y, z
    # points are connected by an edge.
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_trisurf(x, y, z, triangles=tri.triangles, cmap=plt.cm.Spectral)
    ax.set_zlim(-1, 1)

    ax.view_init(30,angle) # Set angle of the plot.

    filename='Images/Image_step'+str(angle)+'.png'
    plt.savefig(filename, dpi=96) # Save the image as a png in Image directory. .
    #plt.gca()
    plt.close(fig=fig) # Deletes figure-objects, to avoid memory leak.
    

print("Finished running the code.")

For at koden ovenfor skal fungerer må du lage en mappe inni samme mappe som du har lagret notebooken med navn "Images". 
Etter å ha kjørt koden ovenfor oppdager du at Images-mappen er full av PNG-bilder. Disse skal deretter settes sammen til en GIF.

Vær oppmerksom på at denne koden overskriver alle filene med samme navn i Images-mappen hver gang du kjører koden. For å endre dette kan du endre mappenavnet i koden fra 'Images' til noe annet, og opprette så mange mapper du ønsker. Tilsvarende endring må dermed gjøres i `ImageMagick`-kommandoen nedenfor.

### Bruk av `ImageMagick`

**Mac / Linux**

Åpne et terminal-vindu og aktiver 'miljøet' 'ImageMagick_env' som du lagde ovenfor. 
Pass på at du befinner deg i rett mappe. For at kommandoen nedenfor skal fungere ønsker du å være i samme mappe som du har lagret denne notebooken i. Bruk kommandoen `cd` (change directory) for å bytte mappe. Du kan finne ut hvor du befinner deg med kommandoen `pwd` og du kan liste mapper og filer der du befinner deg med `ls`.

Eksempel: Dersom du har lagret notebooken din i 'path'-en /Users/moi/termisk/exercise3, må du skrive `cd /Users/avi/termisk/exercise3`, gitt at du befinner deg i rot-mappen. 

Når du er i rett mappe og har aktivert 'ImageMagick_env', kan du skrive inn følgende linje for å gjøre om PNG-bildene i mappa di til en GIF

> convert -delay 10 Images/Image*.png animated_images.gif

**Windows**

Åpne en Powershell Prompt. Om dette er Anaconda sin eller Windows sin standard-Powershell spiller ingen rolle, gitt at nedlastningen av programvaren gikk som den skulle. Pass på at du befinner deg i rett mappe. For at kommandoen nedenfor skal fungere ønsker du å være i samme mappe som du har lagret denne notebooken i. Bruk kommandoen `cd` (change directory) for å bytte mappe. Du kan finne ut hvor du befinner deg med kommandoen `pwd` og du kan liste mapper og filer der du befinner deg med `ls`.

Eksempel: Dersom du har lagret notebeooken din i 'path'-en C:\moi\termisk\exercise3, må du skrive `cd C:\moi\termisk\exercise3`, gitt at du befinner deg på C-disken. 

Når du er i rett mappe kan du skrive inn følgende linje for å gjøre om PNG-bildene i mappa di til en GIF

> magick convert -delay 10 Images\Image*.png animated_images.gif


Vær oppmerksom på at Linux/Mac bruker 'slash', mens Windows bruker 'backslash' i 'path'-en. Dersom du ikke får til å navigere til rett mappe kan du spørre noen eller søke opp det du lurer på. 

`ImageMagick`-kommandoen vi har brukt ovenfor setter sammen alle PNG-filene som begynner med "Images" i Images-mappa til én fil "animated_images.gif". Når kommandoen er kjørt ferdig vil GIF-fila være lagret i samme mappe som denne notebooken. Åpne fila for å se på din nye GIF. 

Dette er bare et eksempel på argumenter du kan gi til `convert` for å endre på hvordan bildene settes sammen til en GIF. `ImageMagick` har en fin [dokumentasjon](https://imagemagick.org/Usage/anim_basics/) som kan brukes for å utforske dette videre. Der kan du få et bedre innblikk i hva de ulike delene av linja ovenfor gjør også.

Fordelen med dette verktøyet er at du kan plotte akkurat hva du vil, for det er hvordan du lagrer PNG-filene som avgjør hva som er med i GIF-en. Deretter kan du bruke `convert` med ulike argumenter for å få GIF-en til å oppføre seg slik du ønsker. 

Dette kan forhåpentligvis komme til nytte en eller annen gang senere. Et viktig poeng er at dette naturligvis kun er et eksempel på programvare som kan brukes til å lage animasjoner og lignende. Det finnes helt sikkert mange andre alternative måter å gjøre det på. Håper det var litt moro på slutten!